# Treinando Redes Neurais Convolucionais usando Lasagne

Nesse tutorial iremos treinar redes neurais convolucionais (CNNs) usando a biblioteca Lasagne, para o problema de classificação de dígitos MNIST.

Esse tipo de rede neural é computacionalmente intenso, em particular rodando em CPUs, que é o caso para esse tutorial. Por isso, nesse tutorial começaremos com redes neurais "tradicionais" (fully-connected), para aprender sobre a biblioteca, e ganhar intuição sobre como treinar os modelos / como selecionar hyper-parametros, etc.

Vamos começar carregando as bibliotecas, e a base de dados

In [ ]:
import theano
import theano.tensor as T
import lasagne
theano.config.floatX = 'float64'

from helpers import mnist_data  # Função para carregar a base MNIST
import matplotlib.pyplot as plt # Para visualizações
import numpy as np

%matplotlib inline

In [ ]:
(x_train, y_train), (x_valid, y_valid), (x_test, y_test) = mnist_data.load_data()

In [ ]:
#Alguns exemplos:

plt.imshow(x_train[0].squeeze(), cmap='Greys', interpolation='nearest')
plt.axis('off')
plt.title('Primeiro digito da base de treinamento')

f, ax = plt.subplots(4,8)
for i in range(4):
    for j in range(8):
        img = x_train[i*8+j].squeeze()
        ax[i,j].imshow(img, cmap='Greys', interpolation='nearest')
        ax[i,j].axis('off')
plt.suptitle('Exemplos da base de treinamento', fontsize=18);

In [ ]:
# Tamanho da base de treinameto
x_train.shape

# Objetivos  gerais
Notamos acima que possuímos 50 mil exemplos para treinamento, onde cada imagem é de tamanho 28x28 e contém um único dígito. 

A tarefa é um problema de classificação entre 10 classes (dígitos de 0 a 9). Vamos criar modelos resolver o problema de estimar a classe correta, isto é, modelos que estimem: 
$$\hat{\textbf{y}} =P(\textbf{y} | X) $$

Onde $\hat{\textbf{y}}$ é um vetor de 10 posições, consistindo na probabilidade da imagem X pertencer a cada uma das 10 classes. Consideramos a decisão do classificador como o dígito mais provável, isto é:


$$ \DeclareMathOperator*{\argmax}{arg\,max} y_\text{pred} = \argmax_i{\hat{y_i}}$$

Para avaliar o classificador, calculamos a taxa de erro como a fração de exemplos que são classificados incorretamente:

$$ \text{error} = \frac{1}{N}\sum_i{ y_\text{pred} \neq y_\text{true}}$$

Vamos começam modelando $P(\textbf{y} | X)$ usando arquiteturas mais simples (regressão logística), e progredindo até treinar redes neurais convolucionais de múltiplas camadas

# Rede neural com apenas 1 camada (i.e. regressão logística)

Como primeiro exercício, vamos criar um modelo com apenas uma camada (a de saída). Esse modelo é equivalente ao modelo de regressão logística que utilizamos ontem, com a diferença que o problema é de múltiplas classes (10 dígitos possíveis)

Ontem, no exemplo com duas classes, usamos apenas um neurônio, com saída entre $[0,1]$ que estimava a probabilidade da classe $y=1$. Para 10 classes, vamos agora considerar 10 saídas $y_i$. Para ser uma distribuição de probabilidates, precisamos que essas saídas satisfaçam as seguintes propriedades:

$$y_i \ge 0 \qquad \sum_i{y_i} = 1$$

Para isso, ao invés de utilizarmos a função sigmoid, utilizamos a função **softmax** na última camada:

$$y_i = \text{softmax}(z_i) = \frac{e^{z_i}}{\sum_k{e^{z_k}}}$$

O uso do função expoente garante que $y_i > 0$, e o denominador garante que as probabilidades serão normalizadas: $\sum_i{y_i} = 1$.

Os passos que vamos tomar para esse exercício são:
1. Definir uma arquitetura 
2. Definir uma função de custo
3. Escolher um algoritmo de otimização
4. Compilar a função de treinamento
5. Chamar a função de treinamento até convergência

Nesse primeiro exercício, não utilizaremos convoluções. Nesse caso, será mais fácil tratar cada exemplo da base de treinamento como um vetor de tamanho 28 * 28 = 784 dimensões.

In [ ]:
x_train_flat = x_train.reshape(50000, 28*28)
x_valid_flat = x_valid.reshape(10000, 28*28)

print 'Tamanho da base de treinamento: ', x_train_flat.shape
print x_train_flat.shape[1]

## Exercício 1.1 - Definição da arquitetura

Nesse exercício vamos criar uma arquitetura, usando lasagne, que contenha:

* Camada de entrada (InputLayer), considerando o tamanho de cada exemplo (número de dimensões em X)
* Camada de saída, fully-connected (DenseLayer), considerando o tamanho do vetor de saída, e usando a não-linearidade softmax

Nota 1: Para detalhes de quais parâmetros passar para a criação das camadas, verificar: http://lasagne.readthedocs.io/en/latest/modules/layers.html (ou os slides da apresentação de hoje)

Nota 2: Vamos treinar o modelo em "batches" que podem ter tamanho variado. Para isso, podemos deixar a primeira dimensão da entrada como "None"

Nota 3: Os parâmetros W e b podem ser inicializados com a função padrão usado pelo Lasagne

In [ ]:
from lasagne.layers import InputLayer, DenseLayer
from lasagne.nonlinearities import softmax

In [ ]:
#Sua solução:

net = {}

net['data'] =  ##coloque aqui a definição da camada de entrada
net['out'] = ##coloque aqui a definição da camada de saída 

In [ ]:
# Verificando os tamanhos de entrada e saída:
assert net['data'].shape == (None, 784), 'Entrada deveria ter dimensão 784, e tamanho de batch variável (None, 784)'
assert net['out'].output_shape == (None, 10)
assert net['out'].nonlinearity == softmax,'Saída deveria utilizar a função softmax'
print('OK')

In [ ]:
#Execute essa célula para ver a solução

%load solutions/cnn_nohid.py

## Exercício 1.2 - Definição da função de custo

Para problemas de classificação, a função de custo apropriada é conhecida como "cross-entropy": 

$$L = - \frac{1}{N}\sum \log P(y | x)$$

Essa é a mesma função de custo que utilizamos na regressão logística de duas classes, com a diferença que agora estamos considerando 10 classes ao invés de uma única saída da rede.

Essa função está implementada no lasagne: ```lasagne.objectives.categorical_crossentropy``` [manual](http://lasagne.readthedocs.io/en/latest/modules/objectives.html#lasagne.objectives.categorical_crossentropy)

Para calcular a função de custo, precisamos:
* Definir uma variável do Theano que representa a entrada (nesse exemplo, vamos usar a variavel que o lasagne criou automaticamente) 
* Obter a saída da Rede ($\hat{\textbf{y}} = P(\textbf{y}|X)$) usando a função ```lasagne.layers.get_output``` [manual](http://lasagne.readthedocs.io/en/latest/modules/layers/helper.html#lasagne.layers.get_output)
* Definir uma variável do Theano que representa a saída desejada ($y_\text{true}$ - vamos chamar de "output_var")
* Calcular o custo (erro) entre a saída da rede e a saída desejada, usando a função ```categorical_crossentropy```


Dica: Como estamos definindo o modelo para ser treinado com vários exemplos ao mesmo tempo, a entrada é uma matrix e a saída esperada é um vetor de inteiros (T.ivector)

Dica 2: A função ```categorical_crossentropy``` também retornará um vetor (o erro referente a cada exemplo de entrada). Precisamos então combinar todos os erros, tomando sua média: ```loss = loss.mean()```

In [ ]:
from lasagne.objectives import categorical_crossentropy

In [ ]:
#Sua solução:

input_var = net['data'].input_var
predicted =  ## Obter a saída da rede

output_var = ## Criação da variável simbólica de saída
loss =  ## Cálculo da função de custo

loss = loss.mean()

In [ ]:
#Execute essa célula para ver a solução

%load solutions/cnn_loss.py

In [ ]:
#Vamos considerar também a taxa de acerto

y_pred = T.argmax(predicted, axis=1)
acc = T.eq(y_pred, output_var)
acc = acc.mean()

## 1.3 - Definição do algoritmo de treinamento

No exemplo de regressão logística, utilizamos o algoritmo de descida de gradiente (Gradient Descent). No entanto, existem outros algoritmos de otimização que convergem mais rápido, como nesterov_momentum e adam. Lasagne possui vários deles implementados: http://lasagne.readthedocs.io/en/latest/modules/updates.html

Nesse exemplo, vamos continuar usando descida de gradiente (em lasagne, chamado ```lasagne.updates.sgd```). Essa função é chamada da seguinte forma:

```
updates = lasagne.updates.sgd(loss, params, lr)
```

Onde os parâmetros são:
* loss: a função de custo a ser otimizada
* parametros: uma lista de parâmetros do modelo
* lr: Learning Rate: o tamanho do passo a ser dado em cada etapa

A saída dessa função (**updates**), é uma lista de pares do tipo [(variavel, expressão)], que define como cada variável é atualizada. Esse método implementa exatamente a mesma função que usamos ontem (por exemplo, se tivermos apenas dois parametros **w** e **b**, a função retorna:

```
updates = [
    (w, w - lr  * w_grad),
    (b, b - lr * b_grad)
]
```

Para obtermos a lista de todos os parâmetros do modelo, utilizamos a função:

```
params = lasagne.layers.get_all_params(ultima_camada)
```

Onde passamos como parâmetro a última camada do modelo, e é retornada uma lista de todos os parâmetros

In [ ]:
lr = 0.01

params = lasagne.layers.get_all_params(net['out'])
updates = lasagne.updates.sgd(loss, params, lr)

## 1.4 - Compilando a função de treinamento

Da mesma forma que no exercício de ontem, compilamos a função de treinamento para retornar o erro na base de treinamento, e também atualizar os parâmetros. Dessa forma, cada chamada à função de treinamento atualiza os parâmetros para reduzir o erro.


In [ ]:
train_fn = theano.function([input_var, output_var], [loss, acc], updates=updates)
val_fn = theano.function([input_var, output_var], [loss, acc])

## Exercício 1.5 - Treinamento

Nesse exercício, vamos fazer o treinamento do modelo defindo acima, chamando a função de treinamento várias vezes, usando todos os dados de treinamento (```x_train_flat, y_train```).

Para monitorar o progresso, vamos também calcular o erro na base de validação (```x_valid_flat, y_valid```), chamando a função de validação (```val_fn```), que retorna o erro, mas não utiliza os dados para alterar o modelo. Vamos também salvar o histórico de custo (em treinamento e validação) para depois monitorarmos o progresso do aprendizado

In [ ]:
#Sua Solução:
cost_history = []   # Insira o custo do treinamento (a cada chamada) nessa lista. 
acc_history = []    # Taxa de acerto do treinamento
val_cost_history = [] # Custo da base de validação
val_acc_history = [] # Taxa de acerto na base de validação

for i in range(50):    
    ## Coloque aqui o código para chamar a função de treinamento (salvando o custo 
    ##   e taxa de acerto de cada chamada), e a chamada à função de validação

In [ ]:
#Execute essa célula para ver a solução

%load solutions/cnn_trainloop.py

## Visualizando a curva de aprendizado

O código acima fez o treinamento do modelo por 50 iterações. Vamos agora visualizar o progresso do treinamento (valor da função de custo ao longo do tempo): 

In [ ]:
plt.plot(cost_history, 'b--', label='Treinamento')
plt.plot(val_cost_history, 'r-', label='Validacao')
plt.xlabel('Numero de iteracoes', fontsize=15)
plt.ylabel('Custo', fontsize=15)
plt.legend()

Notamos que o erro, tanto em treinamento como em validação diminui ao longo do tempo - então o treinamento está funcionando. No entanto, notamos que o erro ainda está diminuindo ao fim das 50 iterações, então poderíamos continuar o treinamento

## Taxa de acerto na base de validação:

In [ ]:
val_acc_history[-1]

# Refatorando o código

No exercício acima, fizemos a definição da arquitetura e todo o código para treinamento usando variáveis globais (em uma única função). Na prática, vamos querer avaliar diferentes tipos de arquiteturas, modificar a função de custo ou o loop de treinamento, então é conveniente separar essas funcionalidades em diferentes métodos.

Vamos considerar três funções:

```
net = build_no_hid_layer()
``` 

> Definição de arquitetura (retorna um dicionário "net" com a definição da rede)

```
train_fn, val_fn = compile_train_function(net, lr)
```

> Dado uma arquitetura e learning rate, compila e retorna a função de treinamento e validação

```
training_curves = train(train_fn, val_fn, train_set, valid_set, epochs)
```

> Dado as funções de treinamento e validação; as bases de treinamento de validação e o número de "epochs" (iterações na base de dados), realiza o treinamento da rede, e retorna as curvas de aprendizado

In [ ]:
def build_no_hid_layer():
    net = {}

    net['data'] = InputLayer((None, 28*28))
    net['out'] = DenseLayer(net['data'], 10, nonlinearity=softmax)
    return net

In [ ]:
def compile_train_function(net, lr):
    input_var = net['data'].input_var
    output_var = T.ivector()

    predicted = lasagne.layers.get_output(net['out'], inputs=input_var)
    loss = lasagne.objectives.categorical_crossentropy(predicted, output_var)
    loss = loss.mean()

    y_pred = T.argmax(predicted, axis=1)
    acc = T.eq(y_pred, output_var)
    acc = acc.mean()

    params = lasagne.layers.get_all_params(net['out'])
    updates = lasagne.updates.sgd(loss, params, lr)

    train_fn = theano.function([input_var, output_var], [loss, acc], updates=updates)
    val_fn = theano.function([input_var, output_var], [loss, acc])
    return train_fn, val_fn

In [ ]:
def train(train_fn, val_fn, train_set, valid_set, epochs):
    x_train, y_train = train_set
    x_valid, y_valid = valid_set
    
    cost_history = []
    acc_history = []
    val_cost_history = []
    val_acc_history = []

    for i in range(epochs):
        cost, acc = train_fn(x_train, y_train)
        cost_history.append(cost)
        acc_history.append(acc)

        val_cost, val_acc = val_fn(x_valid, y_valid)
        val_cost_history.append(val_cost)
        val_acc_history.append(val_acc)
    return cost_history, acc_history, val_cost_history, val_acc_history

In [ ]:
def plot_train_curves(train_curves):
    plt.figure()
    cost_history, acc_history, val_cost_history, val_acc_history = train_curves
    plt.plot(cost_history, 'b--', label='Treinamento')
    plt.plot(val_cost_history, 'r-', label='Validacao')
    plt.xlabel('Numero de iteracoes', fontsize=15)
    plt.ylabel('Custo', fontsize=15)
    plt.legend()
    print('Taxa de acerto em Validação: %.2f%%' % (val_acc_history[-1] * 100))

## Executando o experimento com as funções refatoradas:

In [ ]:
model = build_no_hid_layer()

train_fn, valid_fn = compile_train_function(model, lr=0.01)

train_curves = train(train_fn, valid_fn, 
                     train_set=(x_train_flat, y_train), 
                     valid_set=(x_valid_flat, y_valid),
                     epochs=50)

plot_train_curves(train_curves)

## Exercício 2: Otimizando a Learning Rate

Apesar de o modelo de regressão logística não possuir nenhum hiper-parâmetro, o processo de treinamento usa o hiper-parâmetro "Learning Rate" (tamanho do passo da descida de gradiente). Esse hiper-parâmetro é um dos mais importantes a serem otimizados: se o passo for muito grande, o treinamento pode divergir (i.e. aumentar o erro ao invés de diminuir). Se o passo for muito pequeno, o número de iterações necessárias para convergência aumenta.

Nesse exercício, o objetivo é treinar modelos e comparar as curvas de treinamento usando learning rate variando entre:

$$\text{lr} \in \{0.01, 0.1, 1, 10\}$$

Devido ao custo computacional, vamos limitar o número de iterações (epochs) em 10 nesse exercício.

In [ ]:
#Sua solução: crie 4 modelos, treine cada um por 10 epochs com differentes learning rates, 
#           e use a função plot_train_curves para visualizar as curvas de treinamento de cada uma 

In [ ]:
#Execute essa célula para ver a solução

%load solutions/cnn_lr.py

Notamos que usando $\text{lr} = 0.01$ o modelo demora demais para convergir. No entanto, $\text{lr} = 10$ diverge, e não parece estável. 

As melhores opções estão entre $\text{lr} = 0.1$  e $\text{lr} = 1$. Para o resto dos experimentos, vamos usar o valor de 0.1.

## Stochastic gradient descent

Nos exemplos acima, usamos toda a base de treinamento para cada atualização dos pesos do modelo. Esse método é conhecido como "Batch gradient descent". No entanto, nota-se na prática que se obtem convergência mais rápida usando um algoritmo conhecido como "Stochastic Gradient Descent": ao invés de utilizar todos os exemplos da base de treinamento para calcular a função de custo, e em seguida atualizar os pesos, esse algoritmo utiliza apenas alguns exemplos (um "mini-batch") para atualizar os pesos. Dessa forma, a cada iteração sobre a base de treinamento, a função de custo é avaliada várias vezes, e os pesos atualizados várias vezes.

O código para o treinamento é semelhante ao do Batch Gradient descent, mas nesse caso chamamos a função de treinamento a cada mini-batch de exemplos. É comum definir uma função auxiliar (```iterate_minibatches```) que percorre a base de dados, retornando um mini-batch a cada iteração.

In [ ]:
def iterate_minibatches(x, y, batch_size):
    for batch_start in xrange(0, len(x), batch_size):
        yield x[batch_start:batch_start+batch_size], y[batch_start:batch_start+batch_size]


def train_minibatch(train_fn, val_fn, train_set, valid_set, epochs, batch_size):
    x_train, y_train = train_set
    x_valid, y_valid = valid_set
    
    cost_history = []
    acc_history = []
    val_cost_history = []
    val_acc_history = []
    
    print('epoch\ttrain_err\tval_err')
    for i in range(epochs):
        epoch_cost = 0
        epoch_acc = 0
        train_batches = 0
        for x_batch, y_batch in iterate_minibatches(x_train, y_train, batch_size):
            cost, acc = train_fn(x_batch, y_batch)
            epoch_cost += cost
            epoch_acc += acc
            train_batches += 1

        val_epoch_cost = 0
        val_epoch_acc = 0
        val_batches = 0
        for x_batch, y_batch in iterate_minibatches(x_valid, y_valid, batch_size):
            val_cost, val_acc = val_fn(x_batch, y_batch)
            val_epoch_cost += val_cost
            val_epoch_acc += val_acc
            val_batches += 1
            
        epoch_cost = epoch_cost / train_batches
        cost_history.append(epoch_cost)
        acc_history.append(epoch_acc / train_batches)

        val_epoch_cost = val_epoch_cost / val_batches
        val_cost_history.append(val_epoch_cost)
        val_acc_history.append(val_epoch_acc / val_batches)
        print('%d\t%.4f   \t%.4f' % (i+1, epoch_cost, val_epoch_cost))
    return cost_history, acc_history, val_cost_history, val_acc_history

# Comparando Batch vs Stochastic Gradient Descent

Vamos comparar os dois algoritmos de treinamento, treinando-os por 10 epochs (10 iterações na base de dados)

In [ ]:
import time
start = time.time()

model = build_no_hid_layer()
train_fn, valid_fn = compile_train_function(model, lr=0.1)
train_curves = train(train_fn, valid_fn,     # Treinamento usando Batch Gradient Descent
                     train_set=(x_train_flat, y_train), 
                     valid_set=(x_valid_flat, y_valid),
                     epochs=10)
end = time.time()
plot_train_curves(train_curves)
print("Tempo de treinamento: %.2f segundos" % (end-start))

In [ ]:
start = time.time()
model = build_no_hid_layer()
train_fn, valid_fn = compile_train_function(model, lr=0.1)
train_curves = train_minibatch(train_fn, valid_fn,      # Treinamento usando mini-Batch Gradient Descent
                     train_set=(x_train_flat, y_train), 
                     valid_set=(x_valid_flat, y_valid),
                     epochs=10,
                     batch_size=128)
plot_train_curves(train_curves)
end = time.time()

print("Tempo de treinamento: %.2f segundos" % (end-start))

Podemos ver uma convergência muito mais rápida usando a versão estocástica, usando o mesmo número de iterações

## Exercício 3: Rede Neural com uma camada escondida

Nesse exercício, vamos criar uma rede neural com uma camada escondida (fully-connected). Para tanto, vamos definir três camadas:

* Camada de entrada
* Camada escondida (Dense Layer)
* Camada de saída (Dense Layer com não-linearidade softmax)

Nota: Nesse exerício vamos usar a não-linearidade padrão (ReLU) na camada escondida.

In [ ]:
#Sua solução:

def build_hid_layer(nhid):
    net = {}
    ## Coloque aqui a definição da rede. Use o parametro nhid como o número de neuronios na camada escondida
    return net

In [ ]:
#Execute essa célula para ver a solução

%load solutions/cnn_hid.py

## Exercício 3.2 Otimizando o número de neurônios na camada escondida

Nesse exercício, vamos buscar o melhor número de neurônios na camada escondida.

O objetivo é treinar a rede definida acima usando $\text{nhid} \in \{10, 100, 1000\}$. 
Vamos usar treinamento usando mini-batches, com learning rate 0.1, batch_size=128, e treinando por 30 epochs cada modelo

In [ ]:
#Sua solucao

In [ ]:
#Execute essa célula para ver a solução

%load solutions/cnn_hid_nhid.py

Notamos que os modelos com 100 e 1000 neuronios na camada escondida tem melhor performance (veja que o modelo com 10 neuronios performance muito pior apesar do gráfico ser parecido - note o eixo y) Notamos também que esses modelos começam a entrar em over-fitting - notando que o erro na base de treinamento continua a diminuir, mas o erro na base de validação continua constante.

Notamos também que a performance (98% de acerto já é bem superior à performance sem nenhuma camada escondida (92%)

## Redes Neurais Convolucionais

Vamos considerar agora o treinamento de redes neurais convolucionais.
As mudanças são:
 * A arquitetura da rede, que vai incluir camadas convolucionais
 * A entrada da rede, que terá formato (None, 1, 28, 28) ao invés de (None, 784)
 
O tamanho da entrada (None, 1, 28, 28) segue o padrão do lasagne: a primeira dimensão é o número de exemplos no mini-batch (Usamos "None" para ser variável), a segunda dimensão é o número de canais na imagem (usamos 1 porque a imagem é em escala de cinza; usaríamos 3 para RGB); a terceira e quarta dimensões são altura e largura.

## Exercício 4.1 - Rede convolucional pequena

Vamos começar com uma rede pequena:

* Camada de entrada
* Camada convolucional, com 6 filtros de tamanho 5x5
* Camada de max-pooling, de tamanho 3x3
* Camada de saída, Dense Layer, usando não-linearidade softmax

Dica: Utilize as classes Conv2DLayer e MaxPool2DLayer. http://lasagne.readthedocs.io/en/latest/modules/layers.html

In [ ]:
from lasagne.layers import Conv2DLayer, MaxPool2DLayer

In [ ]:
#Sua solução

def build_conv_small():
    net = {}
    ## Coloque aqui a definição da rede
    
    return net

In [ ]:
#Execute essa célula para ver a solução

%load solutions/cnn_convsmall.py

In [ ]:
model_conv = build_conv_small()
train_fn, valid_fn = compile_train_function(model_conv, lr=0.1)
train_curves = train_minibatch(train_fn, valid_fn, 
                     train_set=(x_train, y_train), 
                     valid_set=(x_valid, y_valid),
                     epochs=30,
                     batch_size=128)
plot_train_curves(train_curves)

Notamos que a performance com a rede convolucional pequena é equivalente à performance usando uma rede com 100 unidades na camada escondida. No entanto, as duas redes possuem  um número bem diferente de parâmetros.

Podemos usar a função ```lasagne.layers.count_params``` para calcular quantos parâmetros são definidos no modelo:

In [ ]:
model_conv = build_conv_small()
lasagne.layers.count_params(model_conv['out'])

In [ ]:
model_hid100 = build_hid_layer(100)
lasagne.layers.count_params(model_hid100['out'])

Notamos então que a rede convolucional obteve a mesma performance usando apenas 4 mil parametros (vs 80 mil parametros no modelo fully-connected)

## Exercício 4.2 - Rede convolucional maior

Nesse exercício, vamos construir uma rede convolucional com mais camadas:

* Camada de entrada
* Camada convolucional, com 8 filtros de tamanho 5x5
* Camada de max-pooling, de tamanho 2x2
* Camada convolucional, com 16 filtros de tamanho 5x5
* Camada de max-pooling, de tamanho 3x3
* Camada fully connected (Dense Layer) com 100 neurônios
* Camada de saída, Dense Layer, usando não-linearidade softmax

Dica: Utilize as classes Conv2DLayer e MaxPool2DLayer. http://lasagne.readthedocs.io/en/latest/modules/layers.html

In [ ]:
#Sua solução:

def build_conv_larger():
    ## Coloque aqui a definição da rede
    
    return net

In [ ]:
#Execute essa celula para ver a solucao

%load solutions/cnn_convlarger.py

Primeiramente, vamos verificar o número de parâmetros nessa rede:

In [ ]:
model_conv_larger = build_conv_larger()
lasagne.layers.count_params(model_conv_larger['out'])

In [ ]:
model_conv_larger = build_conv_larger()
train_fn, valid_fn = compile_train_function(model_conv_larger, lr=0.1)
train_curves = train_minibatch(train_fn, valid_fn, 
                     train_set=(x_train, y_train), 
                     valid_set=(x_valid, y_valid),
                     epochs=30,
                     batch_size=128)
plot_train_curves(train_curves)

Por fim, obtemos uma performance próxima a 99%. Nessa base de dados, redes convolucionais não apresentam resultados tão superiores (o resultado de uma simples rede fully-connected já é bem elevado).




## Salvando e carregando modelos treinados

A biblioteca Lasagne disponibiliza duas funções úteis para salvar / carregar modelos pré-treinados:

```
params = lasagne.layers.get_all_param_values(net['out'])
```
    * Retorna uma lista com o valor de todos os parâmetros do modelo

```
lasagne.layers.set_all_param_values(net['out'], params)
```
    * Atualiza todos os parâmetros do modelo, usando a lista de parametros informada.

Para salvarmos em disco, podemos usar a bibloteca "pickle". Essa biblioteca permite serializar qualquer objeto e escrevê-lo / carregá-lo do disco. 

```
cPickle.dump(variavel, open(nome_de_arquivo, 'w'))
```

    * Serializa e salva a variavel

```
variavel = cPickle.load(open(nome_de_arquivo))
```

    * Carrega uma variável do disco.


Abaixo temos um exemplo completo


In [ ]:
import cPickle 

params = lasagne.layers.get_all_param_values(model_conv_larger['out'])
cPickle.dump(params, open('conv_larger.pickle', 'w'))


In [ ]:
ls -lh 'conv_larger.pickle'

In [ ]:
# Criando uma nova rede:

nova_rede = build_conv_larger()
train_fn, valid_fn = compile_train_function(nova_rede, lr=0.1)

acc = valid_fn(x_valid, y_valid)[1]
print ('Taxa de acerto da nova rede: %.2f' % (acc*100))

Notamos que a "nova_rede" está inicializada aleatoriamente. Vamos agora carregar os parâmetros salvos em disco:

In [ ]:
loaded_params = cPickle.load(open('conv_larger.pickle'))
lasagne.layers.set_all_param_values(nova_rede['out'], loaded_params)

In [ ]:
acc = valid_fn(x_valid, y_valid)[1]
print ('Taxa de acerto da nova rede: %.2f' % (acc*100))

Notamos que agora a nova_rede contém os pesos carregados do disco

# Exercícios extras (avançado)

## A.1) Randomizando os exemplos da base de treinamento

No algoritmo mostrado acima, os exemplos são sempre utilizados na mesma ordem. Em particular, os mini-batches não variam à cada epoch (o exemplo 0 sempre é utilizado junto com os mesmo exemplos 1 à 31 para o cálculo do gradiente). Um mudança simples que melhora a convergência é a de re-ordenar os dados aleatoriamente no início de cada epoch.

Altere a função "iterate_minibatches" para re-ordernar aleatoriamente os dados antes de começar a retornar cada mini-batch. Uma forma de implementar é criando um array de índices, e re-ordenar esse array aleatoriamente:

```
idx = np.arange(len(x))
np.random.shuffle(idx)

x[idx[0:10]], y[idx[0:10]] # retorna 10 elementos de X e y
```


## A.2) Learning Rate Variável

Nesse tutorial, utilizamos uma "learning rate" fixa durante todo o treinamento. É recomendável dimininuir a Learning Rate ao longo do tempo (e.g. começar com 0.1 e terminar em 0.001) (exemplo, nesse [artigo](http://jmlr.org/proceedings/papers/v28/sutskever13.pdf)). Isso é facilmente implementado em lasagne, passando uma variável compartilhada do theano ao invés de um número:

```
lr = theano.shared(1)
updates = lasagne.updates.sgd(loss, params, lr)
```

Que pode ser atualizada a qualquer momento do treinamento, por exemplo:
```
lr.set_value(0.9)
```

Modifique a função de treinamento para modificar a learning rate a cada epoch. 
Dica: Você pode utilizar a função "np.linspace" para obter um valor de learning rate a cada epoch. Exemplo:
```
np.linspace(0.1, 0.001, 10) # Valor inicial, valor final, número de epochs
# retorna: array([ 0.1  ,  0.089,  0.078,  0.067,  0.056,  0.045,  0.034,  0.023,
        0.012,  0.001])
```

## A.3) Early stopping

Nos exemplos acima, fizemos o treinamento por um número fixo de iterações. Nesse método precisamos definir a priori o número de iterações (o que pode ser um problema, ou não). Observamos também, que em alguns casos, a performance da rede deteriora ao longo do tempo (a performance final em validação é pior que a performance em alguma outra epoch de treinamento). Esse segundo problema é mais importante, e fácil de se resolver: basta manter os parametros que obtiveram melhor performance em validacao. Segue pseudo-codigo:

* best_params = None
* best_val_cost = np.inf
* (loop de treinamento)
   * if val_cost < best_val_cost:
      * best_val_cost = val_cost
      * best_params = lasagne.layers.get_all_params_values(last_layer)

Um algoritmo mais complexo, mas que também resolve a questão do número de iterações a serem executadas pode ser encontrado no livro de Deep Learning [link](http://www.deeplearningbook.org/contents/regularization.html): Algoritmo 7.1 na página 247


## A.4) Data augmentation

Uma tática muito utilizada em Deep Learning chama-se "Data-augmentation". Esse termo engloba técnicas para aumentar o tamanho da base de treinamento. Por exemplo, uma tática comum para visão computacional é incluir translações, rotações e mudanças de zoom nas imagens da base de treinamento.

* Para tranlações, pode-se utilizar manipulação de arrays usando numpy
* Para zoom (scaling), pode-se usar a função: scipy.ndimage.zoom
* Para rotação, pode-se usar a função: scipy.ndimage.rotate

Modifique a função de "iterate_minibatches" para incluir tais alterações na base de treinamento (com uma flag para desativar essas alterações para validação / test)

##  Detalhes importantes não cobertos nesse tutorial

* O uso de camadas não-determinísticas (e.g. dropout, batch normalization), requer um cuidado adicional: durante a validação e teste, queremos desabilitar a parte não determinística (que só é importante para treino). No lasagne, isso é feito passando o parametro "deterministic=True" na função "get_outputs". Por exemplo:

```
train_prediction = lasagne.layers.get_output(net['out'])
test_prediction = lasagne.layers.get_output(net['out'], deterministic=True)
```

* Alguma camadas (e.g. batch normalization) possuem parâmetros que não são treináveis (e.g. representam a média e desvio padrão de um neurônio). Portanto, esses parâmetros não devem ser incluídos nos updates de treinamento. Isso é feito passando o parâmetro "trainable=True" na função "get_all_params"
```
params = lasagne.layers.get_all_params(output_layer, trainable=True)
```


## Links Úteis
Curso de CNNs da Stanford: http://cs231n.stanford.edu/syllabus.html

Livro de Deep Learning:
http://www.deeplearningbook.org/

